In [1]:
import pandas as pd
import numpy as np
from itertools import chain
import pickle

In [2]:
input_path = "E:/CS_Master_Degree_UIUC/CS598_DeepLearning_for_Health_Data/Project/paper290/MIMIC data/"
output_path = "E:/CS_Master_Degree_UIUC/CS598_DeepLearning_for_Health_Data/Project/paper290/Processed_data/"

## ADMISSIONS.csv
1. Explore data and check what columns it has.
2. Find out all patients that have at least 2 admissions (at least 2 unique "HADM_ID"). Per Appendix1 of paper,  
   the dataset contains 7,537 patients with 2 or more encounters that they used in their CLOUT model.
3. Some demographic invetistigaiton such as distribution of ethnicity among the above subset of patient

In [3]:
# Read original dataset and explore the columns
admissions = pd.read_csv(input_path + "ADMISSIONS.csv")
col_names = admissions.columns
print(col_names)


# Findout the patient IDs ("SUBJECT_ID") that have at least 2 encounters 
AC = admissions.groupby(['SUBJECT_ID']).HADM_ID.nunique()
subset_id = AC.loc[AC > 1].reset_index().SUBJECT_ID	

# Get the subset of original dataset
admissions_subset = admissions.loc[admissions.SUBJECT_ID.isin(subset_id)].sort_values(by=['SUBJECT_ID','ROW_ID'])
admissions_subset = admissions_subset[['SUBJECT_ID', 'HADM_ID','ETHNICITY','HOSPITAL_EXPIRE_FLAG']]

# Verify if we get the right number of patients
patient_number = len(admissions_subset.SUBJECT_ID.unique())
print("The number of patients that have at least two encounters is: ",patient_number)
print("The number of records in admissions_subset is: ", admissions_subset.shape[0])
admissions_subset.head()

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME',
       'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION',
       'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION',
       'MARITAL_STATUS', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS',
       'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA'],
      dtype='object')
The number of patients that have at least two encounters is:  7537
The number of records in admissions_subset is:  19993


,SUBJECT_ID,HADM_ID,ETHNICITY,HOSPITAL_EXPIRE_FLAG
224,17,194023,WHITE,0
225,17,161087,WHITE,0
229,21,109451,WHITE,0
230,21,111970,WHITE,1
1,23,152223,WHITE,0


In [4]:
### Explore the ethnicity groups in the dataset
patient_ethnicity = pd.DataFrame(admissions_subset.groupby('ETHNICITY').SUBJECT_ID.unique()).reset_index()
patient_ethnicity['ETHNICITY_COUNT'] = patient_ethnicity.apply(lambda x: len(x['SUBJECT_ID']), axis = 1)
print("All ethnicity groups in original dataset: ", patient_ethnicity.ETHNICITY.values)

# Simplify the groups per paper into five basic groups

E_group = ['WHITE','BLACK','ASIAN','HISPANIC']

def Simple_Ethnicity(req, E_group):
    if req ==  'PORTUGUESE':
        return('WHITE')
    for e in E_group:
        if e in req:
            return(e)
    return('OTHER')

patient_ethnicity['ETHNICITY'] = patient_ethnicity.apply(lambda x: Simple_Ethnicity(x['ETHNICITY'], E_group), axis = 1)

patient_ethnicity.head()

All ethnicity groups in original dataset:  ['AMERICAN INDIAN/ALASKA NATIVE'
 'AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE' 'ASIAN'
 'ASIAN - ASIAN INDIAN' 'ASIAN - CAMBODIAN' 'ASIAN - CHINESE'
 'ASIAN - FILIPINO' 'ASIAN - KOREAN' 'ASIAN - OTHER' 'ASIAN - THAI'
 'ASIAN - VIETNAMESE' 'BLACK/AFRICAN' 'BLACK/AFRICAN AMERICAN'
 'BLACK/CAPE VERDEAN' 'BLACK/HAITIAN' 'CARIBBEAN ISLAND'
 'HISPANIC OR LATINO' 'HISPANIC/LATINO - CENTRAL AMERICAN (OTHER)'
 'HISPANIC/LATINO - COLOMBIAN' 'HISPANIC/LATINO - CUBAN'
 'HISPANIC/LATINO - DOMINICAN' 'HISPANIC/LATINO - GUATEMALAN'
 'HISPANIC/LATINO - HONDURAN' 'HISPANIC/LATINO - MEXICAN'
 'HISPANIC/LATINO - PUERTO RICAN' 'HISPANIC/LATINO - SALVADORAN'
 'MIDDLE EASTERN' 'MULTI RACE ETHNICITY'
 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER' 'OTHER'
 'PATIENT DECLINED TO ANSWER' 'PORTUGUESE' 'UNABLE TO OBTAIN'
 'UNKNOWN/NOT SPECIFIED' 'WHITE' 'WHITE - BRAZILIAN'
 'WHITE - EASTERN EUROPEAN' 'WHITE - OTHER EUROPEAN' 'WHITE - RUSSIAN']


,ETHNICITY,SUBJECT_ID,ETHNICITY_COUNT
0,OTHER,"[5782, 14667, 20082, 31755, 40000, 43501]",6
1,OTHER,[42135],1
2,ASIAN,"[94, 191, 203, 299, 605, 907, 986, 1286, 1292,...",163
3,ASIAN,"[2666, 5962, 11043, 28278, 28860, 30650, 31120...",11
4,ASIAN,"[4334, 11171, 40474, 77471]",4


In [5]:
# Summarize the ethnicity group distribution among the patients and compare to Table-4 of Appendix1 of the paper
Ethnicity_table = patient_ethnicity.groupby('ETHNICITY').agg({'ETHNICITY_COUNT':sum})
Ethnicity_table['ETHNICITY_PERCENTAGE'] = np.round(Ethnicity_table['ETHNICITY_COUNT']/Ethnicity_table['ETHNICITY_COUNT'].sum()*100,1)
Ethnicity_table

,ETHNICITY_COUNT,ETHNICITY_PERCENTAGE
ETHNICITY,,
ASIAN,235,3.0
BLACK,885,11.3
HISPANIC,307,3.9
OTHER,665,8.5
WHITE,5736,73.3


In [6]:
Mort_labels = admissions_subset.groupby('SUBJECT_ID').HOSPITAL_EXPIRE_FLAG.max().values.tolist()
print(len(Mort_labels))
Mort_labels[0:6]

7537


[0, 1, 0, 0, 0, 1]

In [7]:
picklefile = "mort_lables.pkl"
with open(output_path + picklefile, "wb") as pkl_wb_obj:
    pickle.dump(Mort_labels, pkl_wb_obj)

## ICD codes
1. datasets include: DIGANOSES_ICD.csv and its dictionary dataset D_DIAGNOSES_ICD.csv.  D_PROCEDURES_ICD.csv.

In [8]:
diag_icd = pd.read_csv(input_path + 'DIAGNOSES_ICD.csv').drop('ROW_ID', axis = 1)
diag_icd_dict = pd.read_csv(input_path + 'D_ICD_DIAGNOSES.csv').drop('ROW_ID', axis = 1)
# prod_icd = pd.read_csv(input_path + 'PROCEDURES_ICD.csv').drop('ROW_ID', axis = 1)
# prod_icd_dict = pd.read_csv(input_path + 'D_ICD_PROCEDURES.csv').drop('ROW_ID', axis = 1)

In [9]:
diag_icd.head()

,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,109,172335,1.0,40301
1,109,172335,2.0,486
2,109,172335,3.0,58281
3,109,172335,4.0,5855
4,109,172335,5.0,4254


In [10]:
diag_icd_dict.head()

,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,01166,TB pneumonia-oth test,"Tuberculous pneumonia [any form], tubercle bac..."
1,01170,TB pneumothorax-unspec,"Tuberculous pneumothorax, unspecified"
2,01171,TB pneumothorax-no exam,"Tuberculous pneumothorax, bacteriological or h..."
3,01172,TB pneumothorx-exam unkn,"Tuberculous pneumothorax, bacteriological or h..."
4,01173,TB pneumothorax-micro dx,"Tuberculous pneumothorax, tubercle bacilli fou..."


In [11]:
diag_icd = diag_icd.merge(diag_icd_dict, on = 'ICD9_CODE' , how='inner')
print(diag_icd.shape[0])
# prod_icd = prod_icd.merge(prod_icd_dict, on = 'ICD9_CODE', how='left')

634709


In [12]:
diag_icd.head()

,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,109,172335,1.0,40301,Mal hyp kid w cr kid V,"Hypertensive chronic kidney disease, malignant..."
1,109,173633,1.0,40301,Mal hyp kid w cr kid V,"Hypertensive chronic kidney disease, malignant..."
2,109,131345,1.0,40301,Mal hyp kid w cr kid V,"Hypertensive chronic kidney disease, malignant..."
3,109,131376,1.0,40301,Mal hyp kid w cr kid V,"Hypertensive chronic kidney disease, malignant..."
4,109,135923,1.0,40301,Mal hyp kid w cr kid V,"Hypertensive chronic kidney disease, malignant..."


In [13]:
icd_codes_all = diag_icd.ICD9_CODE.unique().tolist()
picklefile = "ICD_codes_all.pkl"
with open(output_path + picklefile, "wb") as pkl_wb_obj:
    pickle.dump(icd_codes_all, pkl_wb_obj)

In [14]:
ICD = diag_icd
admissions_subset1 = admissions_subset.merge(ICD, on  = ['SUBJECT_ID','HADM_ID'], how='inner'). \
                    fillna(-1).drop('ETHNICITY', axis=1).sort_values(['SUBJECT_ID','HADM_ID', 'SEQ_NUM'])
print('The volcabulary size of Diagnoses ICD9 codes is: ',admissions_subset1.ICD9_CODE.nunique())
admissions_subset1.head(10)

The volcabulary size of Diagnoses ICD9 codes is:  4780


,SUBJECT_ID,HADM_ID,HOSPITAL_EXPIRE_FLAG,SEQ_NUM,ICD9_CODE,SHORT_TITLE,LONG_TITLE
7,17,161087,0,1.0,4239,Pericardial disease NOS,Unspecified disease of pericardium
4,17,161087,0,2.0,5119,Pleural effusion NOS,Unspecified pleural effusion
9,17,161087,0,3.0,78551,Cardiogenic shock,Cardiogenic shock
8,17,161087,0,4.0,4589,Hypotension NOS,"Hypotension, unspecified"
5,17,161087,0,5.0,311,Depressive disorder NEC,"Depressive disorder, not elsewhere classified"
10,17,161087,0,6.0,7220,Cervical disc displacmnt,Displacement of cervical intervertebral disc w...
11,17,161087,0,7.0,71946,Joint pain-l/leg,"Pain in joint, lower leg"
6,17,161087,0,8.0,2724,Hyperlipidemia NEC/NOS,Other and unspecified hyperlipidemia
2,17,194023,0,1.0,7455,Secundum atrial sept def,Ostium secundum type atrial septal defect
0,17,194023,0,2.0,45829,Iatrogenc hypotnsion NEC,Other iatrogenic hypotension


In [15]:
subjects = admissions_subset1.SUBJECT_ID.unique()
ICDs = []
for subject in subjects:
    temp1 = admissions_subset1.loc[admissions_subset1.SUBJECT_ID == subject]
    encounters = temp1.HADM_ID.unique()
    codes = []
    for e in encounters:
        temp2 = temp1.loc[temp1.HADM_ID == e]
        codes.append(temp2.ICD9_CODE.tolist())
    ICDs.append(codes)

In [16]:
picklefile = "ICD_seq.pkl"
with open(output_path + picklefile, "wb") as pkl_wb_obj:
    pickle.dump(ICDs, pkl_wb_obj)

# #Check if the ICD_seq was saved properly
# with open(output_path + picklefile, "rb") as pkl_rb_obj:
#     seqs = pickle.load(pkl_rb_obj)
# print(seqs)

## Medication
1. Dataset is 'PRESCRIPTIONS.csv' and the column 'DRUG' is the feature for model building

In [17]:
prescriptions = pd.read_csv(input_path + 'PRESCRIPTIONS.csv', low_memory=False).drop(['ROW_ID'],axis = 1)
print(prescriptions.columns.values)

prescriptions.head()

['SUBJECT_ID' 'HADM_ID' 'ICUSTAY_ID' 'STARTDATE' 'ENDDATE' 'DRUG_TYPE'
 'DRUG' 'DRUG_NAME_POE' 'DRUG_NAME_GENERIC' 'FORMULARY_DRUG_CD' 'GSN'
 'NDC' 'PROD_STRENGTH' 'DOSE_VAL_RX' 'DOSE_UNIT_RX' 'FORM_VAL_DISP'
 'FORM_UNIT_DISP' 'ROUTE']


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,469061711.0,1mg Capsule,2,mg,2,CAP,PO
1,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,56017275.0,5mg Tablet,5,mg,1,TAB,PO
2,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,338055002.0,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
3,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.0,HEPARIN BASE,250,ml,250,ml,IV
4,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,54829725.0,20mg Tablet,20,mg,1,TAB,PO


In [18]:
medications = prescriptions[['SUBJECT_ID', 'HADM_ID','DRUG']]
medications.head()

,SUBJECT_ID,HADM_ID,DRUG
0,6,107064,Tacrolimus
1,6,107064,Warfarin
2,6,107064,Heparin Sodium
3,6,107064,D5W
4,6,107064,Furosemide


In [19]:
meds_all = medications.DRUG.unique().tolist()
picklefile = "meds_all.pkl"
with open(output_path + picklefile, "wb") as pkl_wb_obj:
    pickle.dump(meds_all, pkl_wb_obj)

In [20]:
admissions_subset2 = admissions_subset.merge(medications, on = ['SUBJECT_ID','HADM_ID'], how = 'left'). \
                    fillna(-1).drop('ETHNICITY', axis=1).sort_values(['SUBJECT_ID','HADM_ID'])
print('The volcabulary size of Medications is: ',admissions_subset2.DRUG.nunique())
admissions_subset2.head(10)
# admissions_subset3.SUBJECT_ID.nunique() #Check if still have 7537 patients

The volcabulary size of Medications is:  3203


,SUBJECT_ID,HADM_ID,HOSPITAL_EXPIRE_FLAG,DRUG
55,17,161087,0,Glycopyrrolate
56,17,161087,0,Neostigmine
57,17,161087,0,Calcium Gluconate
58,17,161087,0,Magnesium Sulfate
59,17,161087,0,Meperidine
60,17,161087,0,Oxycodone-Acetaminophen
61,17,161087,0,Sucralfate
62,17,161087,0,Acetaminophen
63,17,161087,0,Acetaminophen
64,17,161087,0,Aspirin EC


In [21]:
subjects = admissions_subset2.SUBJECT_ID.unique()
Meds = []
for subject in subjects:
    temp1 = admissions_subset2.loc[admissions_subset2.SUBJECT_ID == subject]
    encounters = temp1.HADM_ID.unique()
    codes = []
    for e in encounters:
        temp2 = temp1.loc[temp1.HADM_ID == e]
        codes.append(temp2.DRUG.tolist())
    Meds.append(codes)

In [22]:
picklefile = "Med_seq.pkl"
with open(output_path + picklefile, "wb") as pkl_wb_obj:
    pickle.dump(Meds, pkl_wb_obj)

# #Check if the Med_seq was saved properly
# with open(output_path + picklefile, "rb") as pkl_rb_obj:
#     seqs = pickle.load(pkl_rb_obj)
# print(seqs)

## Lab
1. Datasets include LABEVENTS.csv and its dictionary dataset D_LABEVNETS.csv.

In [23]:
labevents = pd.read_csv(input_path + 'LABEVENTS.csv').drop('ROW_ID',axis = 1)
d_labitems = pd.read_csv(input_path + 'D_LABITEMS.csv').drop('ROW_ID',axis = 1)
print(labevents.columns.values)
print(d_labitems.columns.values)

['SUBJECT_ID' 'HADM_ID' 'ITEMID' 'CHARTTIME' 'VALUE' 'VALUENUM' 'VALUEUOM'
 'FLAG']
['ITEMID' 'LABEL' 'FLUID' 'CATEGORY' 'LOINC_CODE']


In [24]:
labevents = labevents.merge(d_labitems, on = 'ITEMID', how = 'inner')[['SUBJECT_ID','HADM_ID','ITEMID','LABEL']]

In [25]:
labs_all = labevents.LABEL.unique().tolist()
picklefile = "labs_all.pkl"
with open(output_path + picklefile, "wb") as pkl_wb_obj:
    pickle.dump(labs_all, pkl_wb_obj)

In [26]:
admissions_subset3 = admissions_subset.merge(labevents, on = ['SUBJECT_ID','HADM_ID'], how = 'left').\
                    fillna(-1).drop('ETHNICITY', axis=1).sort_values(['SUBJECT_ID','HADM_ID'])

print('The volcabulary size of Labs is: ',admissions_subset3.LABEL.nunique())
admissions_subset3.head(10)

The volcabulary size of Labs is:  546


,SUBJECT_ID,HADM_ID,HOSPITAL_EXPIRE_FLAG,ITEMID,LABEL
192,17,161087,0,50820.0,pH
193,17,161087,0,50820.0,pH
194,17,161087,0,50820.0,pH
195,17,161087,0,50820.0,pH
196,17,161087,0,50820.0,pH
197,17,161087,0,50820.0,pH
198,17,161087,0,50800.0,SPECIMEN TYPE
199,17,161087,0,50800.0,SPECIMEN TYPE
200,17,161087,0,50800.0,SPECIMEN TYPE
201,17,161087,0,50800.0,SPECIMEN TYPE


In [27]:
subjects = admissions_subset3.SUBJECT_ID.unique()
Labs = []
for subject in subjects:
    temp1 = admissions_subset3.loc[admissions_subset3.SUBJECT_ID == subject]
    encounters = temp1.HADM_ID.unique()
    codes = []
    for e in encounters:
        temp2 = temp1.loc[temp1.HADM_ID == e]
        codes.append(temp2.LABEL.tolist())
    Labs.append(codes)

In [28]:
picklefile = "Lab_seq.pkl"
with open(output_path + picklefile, "wb") as pkl_wb_obj:
    pickle.dump(Labs, pkl_wb_obj)

# #Check if the Med_seq was saved properly
# with open(output_path + picklefile, "rb") as pkl_rb_obj:
#     seqs = pickle.load(pkl_rb_obj)
# print(seqs)

In [29]:
admissions_subset1.to_csv(output_path + 'ICD_dataset.csv', index = False)
admissions_subset2.to_csv(output_path + 'Medication_dataset.csv', index = False)
admissions_subset3.to_csv(output_path + 'Lab_dataset.csv', index = False)